In [1]:
!sudo add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!sudo apt-get update -qq 2>&1 > /dev/null
!sudo apt -y install -qq google-drive-ocamlfuse 2>&1 > /dev/null
!google-drive-ocamlfuse



debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
/usr/bin/xdg-open: 869: www-browser: not found
/usr/bin/xdg-open: 869: links2: not found
/usr/bin/xdg-open: 869: elinks: not found
/usr/bin/xdg-open: 869: links: not found
/usr/bin/xdg-open: 869: lynx: not found
/usr/bin/xdg-open: 869: w3m: not found
xdg-open: no method available for opening 'https://accounts.google.com/o/oauth2/auth?client_id=564921029129.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fgd-ocaml-auth.appspot.com%2Foauth2callback&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&

In [2]:
!sudo apt-get install -qq w3m # to act as web browser 
!xdg-settings set default-web-browser w3m.desktop # to set default browser
%cd /content
!mkdir drive
%cd drive
!mkdir MyDrive
%cd ..
%cd ..
!google-drive-ocamlfuse /content/drive/MyDrive

debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package w3m.
(Reading database ... 122354 files and directories currently installed.)
Preparing to unpack .../w3m_0.5.3-37ubuntu0.1_amd64.deb ...
Unpacking w3m (0.5.3-37ubuntu0.1) ...
Setting up w3m (0.5.3-37ubuntu0.1) ...
Processing triggers for man-db (2.9.1-1) ...
Processing triggers for mime-support (3.64ubuntu1) ...
/content
/content/drive
/content
/
Access token retrieved correctly.


In [3]:
# import the libraries as shown below

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
#import matplotlib.pyplot as plt


In [4]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

train_path = '/content/drive/MyDrive/MIMIA/Covid19-dataset-HistEq-GuassBlur/train'
valid_path = '/content/drive/MyDrive/MIMIA/Covid19-dataset-HistEq-GuassBlur/test'

In [5]:
# Import the VGG16 library as shown below and add preprocessing layer to the front of VGG
# Here we will be using imagenet weights

vgg16 = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

58889256/58889256 [==============================] - 3s 0us/step


In [6]:
# don't train existing weights
for layer in vgg16.layers:
    layer.trainable = False


In [7]:
# our layers - you can add more if you want
x = Flatten()(vgg16.output)

In [8]:
prediction = Dense(3, activation='softmax')(x)

# create a model object
model = Model(inputs=vgg16.input, outputs=prediction)

In [9]:
# view the structure of the model
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [10]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [11]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [12]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/MIMIA/Covid19-dataset-HistEq-GuassBlur/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 251 images belonging to 3 classes.


In [13]:
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/MIMIA/Covid19-dataset-HistEq-GuassBlur/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 66 images belonging to 3 classes.


In [14]:
# fit the model
# Run the cell. It will take some time to execute
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=5,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

<ipython-input-14-6ec5a35725ca>:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model.fit_generator(


Epoch 1/5
8/8 [==============================] - 333s 40s/step - loss: 1.4007 - accuracy: 0.5299 - val_loss: 0.6529 - val_accuracy: 0.6515
Epoch 2/5
8/8 [==============================] - 15s 2s/step - loss: 0.4963 - accuracy: 0.7849 - val_loss: 0.5187 - val_accuracy: 0.7424
Epoch 3/5
8/8 [==============================] - 14s 2s/step - loss: 0.2503 - accuracy: 0.9084 - val_loss: 0.2447 - val_accuracy: 0.9242
Epoch 4/5
8/8 [==============================] - 14s 2s/step - loss: 0.1461 - accuracy: 0.9602 - val_loss: 0.2655 - val_accuracy: 0.8636
Epoch 5/5
8/8 [==============================] - 16s 2s/step - loss: 0.1314 - accuracy: 0.9522 - val_loss: 0.1643 - val_accuracy: 0.9545


In [15]:
# fit the model
# Run the cell. It will take some time to execute
r1 = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=10,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

<ipython-input-15-b4bf0cc3fc21>:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r1 = model.fit_generator(


Epoch 1/10
8/8 [==============================] - 16s 2s/step - loss: 0.1076 - accuracy: 0.9602 - val_loss: 0.1537 - val_accuracy: 0.9697
Epoch 2/10
8/8 [==============================] - 14s 2s/step - loss: 0.0884 - accuracy: 0.9721 - val_loss: 0.1462 - val_accuracy: 0.9545
Epoch 3/10
8/8 [==============================] - 14s 2s/step - loss: 0.0967 - accuracy: 0.9801 - val_loss: 0.1351 - val_accuracy: 0.9697
Epoch 4/10
8/8 [==============================] - 15s 2s/step - loss: 0.0755 - accuracy: 0.9801 - val_loss: 0.1386 - val_accuracy: 0.9697
Epoch 5/10
8/8 [==============================] - 14s 2s/step - loss: 0.0819 - accuracy: 0.9721 - val_loss: 0.1338 - val_accuracy: 0.9545
Epoch 6/10
8/8 [==============================] - 14s 2s/step - loss: 0.0655 - accuracy: 0.9880 - val_loss: 0.1330 - val_accuracy: 0.9697
Epoch 7/10
8/8 [==============================] - 14s 2s/step - loss: 0.0617 - accuracy: 0.9880 - val_loss: 0.1203 - val_accuracy: 0.9697
Epoch 8/10
8/8 [==================